In [30]:
import pandas as pd
import numpy as np

In [31]:
data4 = pd.read_csv('data/train.tsv', sep='\t')

In [32]:
data4.dropna(subset=["category_name"], inplace=True) # 1번

In [33]:
data4["item_description"] = data4["item_description"].fillna("No description") # 1번

In [34]:
data4 = data4[data4['price']!=0] # 2 번 

In [35]:
data4[['cat1','cat2', 'cat3']] = data4['category_name'].str.split("/",expand = True, n=2) # 3번

In [36]:
data4 = data4.applymap(lambda x: x.lower() if isinstance(x, str) else x) # 4번

C:\Users\yongr\AppData\Local\Temp\ipykernel_22524\2005271090.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data4 = data4.applymap(lambda x: x.lower() if isinstance(x, str) else x) # 4번


In [37]:
import re

# 브랜드 키워드 사전
brands = [
    "nike","adidas","puma","reebok","under armour", "mlb"
    "victoria's secret","lularoe","razer","target",
    "gucci","prada","chanel","fendi","versace","zara","h&m"
]
# 특수문자 처리
pattern = "(" + "|".join(map(re.escape, brands)) + ")"

# 결측치 선택 후 브랜드 추출 코드
mask = data4["brand_name"].isna()
hits = (
    data4.loc[mask,"name"].str.extract(pattern, expand=False)
    .fillna(data4.loc[mask,"item_description"].str.extract(pattern, expand=False))
)
# 추출한 브랜드를 결측치에 넣는 코드
data4.loc[mask, "brand_name"] = hits

In [38]:
data4["brand_name"] = data4["brand_name"].fillna("no_brand")

In [39]:
data4.isnull().sum()

train_id             0
name                 0
item_condition_id    0
category_name        0
brand_name           0
price                0
shipping             0
item_description     0
cat1                 0
cat2                 0
cat3                 0
dtype: int64

In [40]:
data4['cat3'].value_counts(dropna=False)

cat3
t-shirts                   61444
pants, tights, leggings    60152
other                      50198
face                       50145
shoes                      32138
                           ...  
dishwashers                    1
frames                         1
aceo                           1
tiles                          1
ephemera                       1
Name: count, Length: 870, dtype: int64

In [41]:
# 'cat2' 중분류 그룹화
grouped_by_cat2 = data4.groupby('cat2')

In [42]:
# 이상치 탐지 함수
def detect_outliers(group):
    Q1 = group['price'].quantile(0.25)
    Q3 = group['price'].quantile(0.75)
    IQR = Q3 - Q1
    upper_bound = Q3 + 1.5 * IQR

    outliers = group[group['price'] > upper_bound]
    return outliers

In [43]:
# 이상치 탐지 적용 및추출
outlier_row = grouped_by_cat2.apply(detect_outliers)

C:\Users\yongr\AppData\Local\Temp\ipykernel_22524\3063496064.py:2: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  outlier_row = grouped_by_cat2.apply(detect_outliers)


In [44]:
display(outlier_row)

train_id                                     name  \
cat2                                                                    
accessories 3143        3143                                  ray ban   
            12379      12379          bundle deal 4 bundles sale sale   
            13345      13345         luxurious hair 12,14,16,18inches   
            15461      15461                      20" dark brown hair   
            18308      18308                        rayban eyeglasses   
...                      ...                                      ...   
woodworking 431367    431367                                 for ryan   
            615262    615262          usc handcrafted wooden football   
            846240    846240  mt. rainier 100th anniversary for susan   
            862928    862928            2 bordered monograms for ford   
            927132    927132                   reserved - yllasmoreno   

                    item_condition_id  \
cat2                                    
accessories 3143                    3   
            12379                   1   
            13345                   1   
            15461                   1   
            18308                   2   
...                               ...   
woodworking 431367                  1   
            615262                  1   
            846240                  1   
            862928                  1   
            927132                  1   

                                                 category_name brand_name  \
cat2                                                                        
accessories 3143    vintage & collectibles/accessories/eyewear    ray-ban   
            12379                    handmade/accessories/hair   no_brand   
            13345                    handmade/accessories/hair   no_brand   
            15461                    handmade/accessories/hair   no_brand   
            18308   vintage & collectibles/accessories/eyewear    ray-ban   
...                                                        ...        ...   
woodworking 431367                  handmade/woodworking/signs   no_brand   
            615262                 handmade/woodworking/sports   no_brand   
            846240                  handmade/woodworking/signs   no_brand   
            862928                  handmade/woodworking/signs   no_brand   
            927132                  handmade/woodworking/signs   no_brand   

                    price  shipping  \
cat2                                  
accessories 3143     54.0         0   
            12379   105.0         1   
            13345   171.0         0   
            15461    81.0         0   
            18308    85.0         1   
...                   ...       ...   
woodworking 431367  120.0         1   
            615262   86.0         0   
            846240   95.0         1   
            862928   49.0         0   
            927132   57.0         0   

                                                     item_description  \
cat2                                                                    
accessories 3143    ray ban eyeglasses 52-18 145 wayfarer this fra...   
            12379   4 bundles virgin hair 18 20. 22 24 free eye la...   
            13345   if u have no ratings dont purchase! 4 bundles ...   
            15461   20" dark brown clip ins 7 pieces 100% human ha...   
            18308   authentic rayban eyeglasses size 55-18-145 ori...   
...                                                               ...   
woodworking 431367                            5 cross country runners   
            615262  this beautiful handcrafted wooden football is ...   
            846240                           custom listing for susan   
            862928                                 no description yet   
            927132  handmade wood signs 1. song of solomon (14"x18...   

                                      cat1         cat2     cat3  
cat2     

In [45]:
# 이상치 행 번호 추출 후 저장
outlier_indices = outlier_row.index.get_level_values(1)
display(outlier_indices)
len(outlier_indices)

Index([   3143,   12379,   13345,   15461,   18308,   19915,   22190,   22406,
         26365,   27338,
       ...
       1481840, 1481940, 1482364,  183825,  253129,  431367,  615262,  846240,
        862928,  927132],
      dtype='int64', length=103657)

103657

In [46]:
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd

# 1) 베이스셋
df = data4.drop(index=outlier_indices, errors="ignore").copy()
df = df[df["price"] > 0].copy()

# ⬇️ 원본 인덱스 보존
df["orig_index"] = df.index

# 파생
df["log_price"] = np.log1p(df["price"])
df["condition_score"] = 6 - df["item_condition_id"].astype(int)

# 2) 희소 묶기 (other 생성 → 삭제)
def lump(s, min_count=50, other="other"):
    vc = s.value_counts()
    return s.where(~s.isin(vc[vc < min_count].index), other)

df["brand_grp"] = lump(df["brand_name"].fillna("no_brand"), 50)
df["cat2_grp"]  = lump(df["cat2"], 50)
df = df[(df["brand_grp"] != "other") & (df["cat2_grp"] != "other")].copy()

# 3) 등급화
def make_grade_pct(df, by, target="log_price", k=20, out="grade", min_n=None):
    gmean = df[target].mean()
    grp = df.groupby(by, dropna=False)[target].agg(count="count", mean="mean").reset_index()
    grp["sm"] = (grp["count"]*grp["mean"] + k*gmean) / (grp["count"] + k)
    grp["prank"] = grp["sm"].rank(pct=True, ascending=True, method="average")
    grp[out] = pd.cut(grp["prank"],
                      bins=[0, 0.25, 0.50, 0.75, 0.90, 1.00],
                      labels=[5, 4, 3, 2, 1],
                      include_lowest=True, right=True).astype("Int64")
    if min_n is not None:
        grp.loc[grp["count"] < min_n, out] = 3
    grp[out] = grp[out].astype(int)
    return df.merge(grp[by + [out]], on=by, how="left", validate="m:1")

df = make_grade_pct(df, ["brand_grp"],       out="brand_grade",     k=20, min_n=20)
df = make_grade_pct(df, ["cat2_grp"],        out="category_grade",  k=20, min_n=10)
df = make_grade_pct(df, ["condition_score"], out="condition_grade", k=10, min_n=None)

# 4) 가중치 계산 → grade_score
from sklearn.linear_model import LinearRegression
X = df[["brand_grade", "category_grade", "condition_grade"]]
y = df["log_price"]
coefs = LinearRegression().fit(X, y).coef_
weights = np.abs(coefs) / np.abs(coefs).sum()
scaled_weights = weights / weights.min()
w1, w2, w3 = scaled_weights
df["grade_score"] = w1*df["brand_grade"] + w2*df["category_grade"] + w3*df["condition_grade"]

# ⬇️ 여기서 원본 인덱스로 되돌리기 (핵심)
df = df.set_index("orig_index")

# 5) data4에 결과 붙이기
cols_to_add = [
    "log_price", "condition_score",
    "brand_grp", "cat2_grp",
    "brand_grade", "category_grade", "condition_grade",
    "grade_score"
]
data4 = data4.join(df[cols_to_add], how="left")


In [47]:
data4

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,cat1,cat2,cat3,log_price,condition_score,brand_grp,cat2_grp,brand_grade,category_grade,condition_grade,grade_score
0,0,mlb cincinnati reds t shirt size xl,3,men/tops/t-shirts,no_brand,10.0,1,no description yet,men,tops,t-shirts,2.397895,3.0,no_brand,tops,5.0,4.0,3.0,167.150850
1,1,razer blackwidow chroma keyboard,3,electronics/computers & tablets/components & p...,razer,52.0,0,this keyboard is in great condition and works ...,electronics,computers & tablets,components & parts,3.970292,3.0,razer,computers & tablets,1.0,1.0,3.0,39.189340
2,2,ava-viv blouse,1,women/tops & blouses/blouse,target,10.0,1,adorable top with a hint of lace and a key hol...,women,tops & blouses,blouse,2.397895,5.0,target,tops & blouses,5.0,4.0,4.0,168.150850
3,3,leather horse statues,1,home/home décor/home décor accents,no_brand,35.0,1,new with tags. leather horses. retail for [rm]...,home,home décor,home décor accents,3.583519,5.0,no_brand,home décor,5.0,3.0,4.0,151.355000
4,4,24k gold plated rose,1,women/jewelry/necklaces,no_brand,44.0,0,complete with certificate of authenticity,women,jewelry,necklaces,3.806662,5.0,no_brand,jewelry,5.0,5.0,4.0,184.946701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1482530,1482530,free people inspired dress,2,women/dresses/mid-calf,free people,20.0,1,"lace, says size small but fits medium perfectl...",women,dresses,mid-calf,3.044522,4.0,free people,dresses,2.0,2.0,1.0,73.378680
1482531,1482531,little mermaid handmade dress,2,kids/girls 2t-5t/dresses,disney,14.0,0,little mermaid handmade dress never worn size 2t,kids,girls 2t-5t,dresses,2.708050,4.0,disney,girls 2t-5t,5.0,4.0,1.0,165.150850
1482532,1482532,21 day fix containers and eating plan,2,sports & outdoors/exercise/fitness accessories,no_brand,12.0,0,"used once or twice, still in great shape.",sports & outdoors,exercise,fitness accessories,2.564949,4.0,no_brand,exercise,5.0,3.0,1.0,148.355000
1482533,1482533,world markets lanterns,3,home/home décor/home décor accents,no_brand,45.0,1,there is 2 of each one that you see! so 2 red ...,home,home décor,home décor accents,3.828641,3.0,no_brand,home décor,5.0,3.0,3.0,150.355000
